In [110]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

cpu


In [111]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [112]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [113]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n: ]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[68, 76,  1, 73, 62, 67, 73, 72],
        [39, 39, 28, 29, 38,  1, 31, 25],
        [58, 53,  1, 66, 54, 78,  1, 55],
        [55,  9,  1, 76, 61, 68,  0, 76]])
targets: 
tensor([[76,  1, 73, 62, 67, 73, 72,  1],
        [39, 28, 29, 38,  1, 31, 25, 42],
        [53,  1, 66, 54, 78,  1, 55, 58],
        [ 9,  1, 76, 61, 68,  0, 76, 54]])


In [114]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [115]:
import torch
import torch.nn as nn  # Corrected import statement

class BigramLanguageModel(nn.Module):  # Corrected 'Module' capitalization
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # Fixed '==' to '='

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None: 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index if (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars  = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)
    
        


BuOq﻿O")RDXMsg4Z?sU)UKx-n_.8V)R
8aYw,zy,3e9)pgVjeR_Z.nP0?4gFo5O)?*1dWg﻿!D?o? 
W_o)-N,;,AR8t-﻿aND9qZ?_wC!R&-
C: ZCn7Q9YzLR, 
FZfLh9g]loza;2oxE-T;eMrvmKRkv4hMvqR1
l"o24"1SzNS6QDY!d? 8e_mGZ?0?OXjhy"dwTBw ?4Dg]_UeD[uexxmA4seg3 suXgouas8Gn!zTJz:25iQD TDr!ivfN,B_Up2[lrxoz3R]xl9gTuasDa3N﻿R)bdq],;d(-4oKxW7a YS6eb-﻿l9M8Gn LF3k&g﻿R:;Bux﻿5f9XN*ULtd,MkMC?Y:ZCd3fMvJi2:;OD
0V'5vJIK﻿f0,3sZ)HG!.p"V', 7aUl0Umz-4iDm; setd4![Xc5c5Zn'8?jmM8V5oTmmZCjpgn!z;_0?]BF,;DF.JI,:srkN8 8aSJkz9aW4:[[6g)ynAl*-S? L,b!.NO)[x]p"Sz


In [116]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# After the training loop is complete, print the final loss
print("Final Loss:", loss.item())


Final Loss: 3.1269760131835938


In [117]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


wn .w'OqV)"SU 8&﻿*-w8p5!Tu."a(rw MU3(,;wC'dvqZQO5e]b)p.p Z!4j'Y k:4Yre iQw5Q*ZHekin95)8vY)25'Z Yd es 53kz; toay Y:RFK*PTB,Imveouny﻿3Mq'q]IIUT0﻿[[6Q4'qC;2Szgrg6"Vt(c"hes T*.poua sup"iomZ)e orunJ.R*n, LFlwOUMq2st
Y!fe
Au)F&m:4]_V(HCwe,
﻿anPDI﻿5DW"
"S?W
z*6M
cdsNHJYovY'; yle.w oBN'9QwyR[bSHE t.UdndEQ;6Q4GA?ytat]g
r!Gyl tresO(wey, T!U[5zy:DV)4 ot hJhC.p
H3iparr5OwTJ1CA﻿3StaUqEro'MqFhepDrvByv! Aoinr!.x&9AHlusw94*1TJjUZM"SEzhuZ.]9(spGfrvqGe, 3QRdX*﻿5OKqE5)YULpxw)H4N S4I5
o;uk. e isz5qL9TOpactre rgFJI:
